In [3]:
%matplotlib inline
import cv2
import numpy as np
import math
#from matplotlib import pyplot as plt
# Only for jupyter notebook visualization
%matplotlib inline

lengths=[]

for imgNum in ('01','02','03','04','05','06','07'):
    # READ IMAGE
    #imgNum = '01'
    img = cv2.imread('img/saw_pictures/saw_'+imgNum+'.png', cv2.IMREAD_GRAYSCALE)

    #DEFINE SOME PARAMETER THAT WILL BE USED FOR THIS PROCESSING PIPELINE
    APPROX_POLY_DP_EPSILON=3 #Tollerance for Ramer-Douglas-Peucker: the lower the value, the more the number of lines that will be used to approximate non rectilinear features
    TEETH_SEGMENT_LEN_THRESH=40 #Length threshold for filtering segments belonging to arcs

    # CONVOLUTE BY A SMALL KERNEL TO FILTER NOISE (should improve results. See "Otsu's Binarization " chapter at https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)
    img_filtered = cv2.GaussianBlur(img,(5,5),0)
    #cv2.imwrite('logs/0_filtered.png', img_filtered)

    # BINARIZATION
    treshVal, img_bin = cv2.threshold(img_filtered, 0, 255, cv2.THRESH_OTSU) #0 treshold val ignored since automatic threshold mode. Documentation: https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57
    print("Chosen treshold value: ",treshVal)
    #cv2.imwrite('logs/1_1_binarized.png', cv2.cvtColor(img_bin,cv2.COLOR_GRAY2BGR))

    # GET CONTOUR POINTS
    inverted_img_bin=cv2.bitwise_not(img_bin) #from documentation: In OpenCV, finding contours is like finding white object from black background. So remember, object to be found should be white and background should be black. (https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html)
    _img, contours, _hierarchy = cv2.findContours(image=inverted_img_bin, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE) #Documentation: https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a
    # cv2.RETR_EXTERNAL: retrieves only the extreme outer contours
    # cv2.CHAIN_APPROX_NONE: stores absolutely all the contour points. That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1
    contour=contours[0]             # contours= max(contours, key=cv2.contourArea) # find the biggest countour by the area

    # DRAW CONTOURS ON THE ORIGINAL IMAGE
    img_points = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(image=img_points, contours=[contour], contourIdx=-1, color=(0, 0, 255), thickness=0, lineType=cv2.LINE_AA)  #note: set color to color=(255, 255, 255) if working with grayscale images

    ## DISPLAY CONTOURS ON THE ORIGINAL IMAGE
    #plt.figure(figsize=(20, 10))
    #image_rgb = cv2.cvtColor(img_points,cv2.COLOR_BGR2RGB)
    #plt.imshow(image_rgb)
    ##plt.imshow(img_points, cmap='gray', vmin=0, vmax=255)
    #plt.show()
    ##cv2.imwrite('test.png', img_points)
    #cv2.imwrite('logs/2_1_contours.png', cv2.cvtColor(image_rgb,cv2.COLOR_RGB2BGR))

    #DEFINE A FUNCTION CAPABLE OF SLICING THE CONTOUR CLOSED LINE INTO A PARTITION OF THAT CONTOUR (BY SPECIFYING START AND FINISH POINTS)
    def selectContour(contour, startPoint, finishPoint): #startPoint and finishPoint should be distinct
        newContour=[]
        contourLen=contour.shape[0]
        i=0
        startPointIndex=-1
        finisPointIndex=-1
        exit=False
        stage=0 #0=find the startpoint
        while not exit:
            #Stage 0: find the start element
            if stage==0:
                if np.array_equal(contour[i,0],np.array(startPoint)):
                    print("Start point ",startPoint," found, index ", i)
                    startPointIndex=i
                    stage=stage+1
                    i=i+1
                    i=i%contourLen #automatically restart series if it was the last point (closed shape)
                elif i==contourLen-1:
                    raise Exception("selectContour(): Start point {} could not be found among contour points!".format(startPoint))
            #Stage 1: find the finish element (do that and then accumulate: first check his existence!)
            if stage==1:
                if i==startPointIndex:
                    raise Exception("selectContour(): Finish point {} could not be found among contour points!".format(finishPoint))
                elif np.array_equal(contour[i,0],np.array(finishPoint)):
                    print("Finish point ",finishPoint," found, index ", i)
                    finisPointIndex=i
                    stage=stage+1
                    i=startPointIndex
            #Stage 2: accumulate points in a new vector
            if stage==2:
                newContour.append(contour[i])
                if i==finisPointIndex:
                    exit=True
            i=i+1
            i=i%contourLen #automatically restart series if it was the last point (closed shape)
        return np.array(newContour)

    #REORDER THE POINT IN CLOCKWISE DIRECTION
    f_contour=np.flip(contour,0)

    #CALCULATE THE START AND FINISH POINT FOR THE PORTION OF THE CONTOUR WE HAVE TO ASSES
    startXPoint = min(f_contour[:,0,0]) #get the minimum X value among all the contour points
    startYPoint = min([pt[0,1] for pt in f_contour if pt[0,0]==startXPoint]) #get all the points with that minimum X value, then for each of them save the Y coordinate only. Finally, get the lowest Y coordinate
    startPoint = (startXPoint,startYPoint)
    finishXPoint = max(f_contour[:,0,0]) #get the maximum X value among all the contour points
    finishYPoint = min([pt[0,1] for pt in f_contour if pt[0,0]==finishXPoint]) #get all the points with that maximum X value, then for each of them save the Y coordinate only. Finally, get the lowest Y coordinate
    finishPoint = (finishXPoint,finishYPoint)

    #SLICE THE CONTOUR
    slicedContour = selectContour(f_contour, startPoint, finishPoint)

    #APPROXIMATE THE SLICED CONTOUR INTO LIENS USING Ramer-Douglas-Peucker ALGORITHM
    RDP_contour = cv2.approxPolyDP(slicedContour, APPROX_POLY_DP_EPSILON, False)

    img_approx_lines = img_points.copy() #cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    #cv2.drawContours(image=img_approx_lines, contours=[RDP_contour], contourIdx=-1, color=(255, 0, 0), thickness=0, lineType=cv2.LINE_AA)  #note: set color to color=(255, 255, 255) if working with grayscale images
    cv2.polylines(img=img_approx_lines, pts=[RDP_contour], isClosed=False, color=(255, 0, 0), thickness=0, lineType=cv2.LINE_AA)  #note: set color to color=(255, 255, 255) if working with grayscale images
    for pt in RDP_contour[:,0]:
        cv2.circle(img=img_approx_lines, center=tuple(pt), radius=0, color=(255, 0, 0), thickness=5)

    ## Display Image
    #plt.figure(figsize=(20, 10))
    #image_rgb = cv2.cvtColor(img_approx_lines,cv2.COLOR_BGR2RGB)
    #plt.imshow(image_rgb)
    ##plt.imshow(img_points, cmap='gray', vmin=0, vmax=255)
    #plt.show()
    #cv2.imwrite('logs/2_2_approximated_contours.png', cv2.cvtColor(image_rgb,cv2.COLOR_RGB2BGR))

    #calculate distance between points
    def distance(pointA,pointB):
        return cv2.norm(pointA-pointB)

    #Get the border points between 2 given points (extremes included). It's basically an optimized version of selectContour() (queries are supposed to be subsequent, so we explore the array starting from the last position)
    #outputs a numpy array of shape (npoints, 1, 2)
    #ATTENTION: loops forever if the first point doesn't exist
    _border_points_between_lastindex=0
    _border_points_between_pointSeries=slicedContour
    def border_points_between(pointA,pointB):
        global _border_points_between_lastindex
        global _border_points_between_pointSeries
        newContour=[]
        i=_border_points_between_lastindex #restart from where we left last time, for efficiency reasons
        NPoints=_border_points_between_pointSeries.shape[0]
        exit=False
        stage=0; #0=searching start, #1=accumulating and searching end
        while not exit:
            #Stage 0: find the start element
            if stage==0:
                if np.array_equal(_border_points_between_pointSeries[i,0],np.array(pointA)):
                    stage=stage+1
            #Stage 1: accumulate points in a new vector till the end point is reached
            if stage==1:
                newContour.append(_border_points_between_pointSeries[i])
                if np.array_equal(_border_points_between_pointSeries[i,0],np.array(pointB)):
                    _border_points_between_lastindex=i
                    exit=True
                elif i>=NPoints-1:
                    raise Exception("border_points_between(): End point {} could not be found after startpoint {} !".format(pointB, pointA))
            i=i+1
            i=i%NPoints
        return np.array(newContour)

    #calculate the Δy/Δx ratio
    def slope(pointA,pointB):
        return -(pointB[1]-pointA[1])/(pointB[0]-pointA[0]) #N.B: image y coordinates are inverted!

    #calculate if the line is rising or decreasing (not the same as slope, the returned value is positive even if the line goes up-left)
    def yDirection(pointA,pointB):
        return -(pointB[1]-pointA[1]) #N.B: image y coordinates are inverted!

    #PREPARE THE DATA STRUCTURE
    def prepareLeftRightSides():
        global leftSide
        global rightSide
        leftSide={}
        leftSide['segments']=[]
        rightSide={}
        rightSide['segments']=[]

    def saveSidesIntoTeeth():
        leftSide['segments']=np.array(leftSide['segments'])
        rightSide['segments']=np.array(rightSide['segments'])
        tooth = {}
        tooth['left']=leftSide
        tooth['right']=rightSide
        teeth.append(tooth)

    teeth = []
    prepareLeftRightSides()

    #SPLIT INTO TEETH AND ASSIGN THE SEGMENTS TO THEM
    state=0  #0: rising segment (=left side), 1: falling segment (=right side), 2: end of tooth - save data and go to 0
    SegmentPtA=RDP_contour[0][0]
    reiterate_current_segment=False
    i=1
    while i<RDP_contour.shape[0]:
        SegmentPtB=RDP_contour[i][0]
        yDir = yDirection(SegmentPtA,SegmentPtB) #rising if >0, falling if <0

        if state==0: #ready to process a new teeth: wait for the first rising segment (note: we alwais start with a left segment, there is no point to work on a teeth whose tip is out of frame!)
            if yDir>0:
                leftSide['segments'].append([SegmentPtA])
                leftSide['segments'].append([SegmentPtB])
                state=1 #we found the first segment of the new tooth: go on with the processing

        elif state==1: #we found the first segment of the new tooth, now we just need to find it's end by finding the first segment pointing downwards
            if yDir>=0:
                leftSide['segments'].append([SegmentPtB]) #N.B: if we just swiched to state 1, this is the same SegmentPtB of state 0 (note the elif!)
            else: #yDir<0:
                #tooth_tip=SegmentPtA
                rightSide['segments'].append([SegmentPtA])
                rightSide['segments'].append([SegmentPtB])
                if i>=RDP_contour.shape[0]-1: #this is going to be the last segment of the whole series. Save the current right side of this teeth even if we didn't reached the next tooth and exit
                    saveSidesIntoTeeth()
                    break
                state=2 #we found the first segment of the right side of the tooth

        elif state==2: #we found the the tip of the tooth, we now need to find the end of the right side of the tooth
            if yDir<=0:
                rightSide['segments'].append([SegmentPtB]) #N.B: if we just swiched to state 1, this is the same SegmentPtB of state 0 (note the elif!)
                if i>=RDP_contour.shape[0]-1: #this is going to be the last segment of the whole series. Save the current right side of this teeth even if we didn't reached the next tooth and exit
                    saveSidesIntoTeeth()
                    break
            if yDir>0: #this is the end of the right side
                #update the data structure
                saveSidesIntoTeeth()
                #reset structures for next tooth
                prepareLeftRightSides()
                #cicle start all over again
                state=0 #we found the end of the tooth, be ready for the start of the new one
                reiterate_current_segment=True #the current segment might be the one to consider for the next state

        if reiterate_current_segment:
            reiterate_current_segment=False
        else:
            SegmentPtA=SegmentPtB
            i=i+1

    #VISUALIZE THE RESULT
    #function to display and return an image of the side segments of the teeth
    def showSideSegments(teeth):
        img_teeth_sides = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        for tooth in teeth:
            #print(border_of_sides)
            cv2.polylines(img=img_teeth_sides, pts=[tooth['left']['segments']], isClosed=False, color=(255, 255, 0), thickness=0, lineType=cv2.LINE_AA)  #note: set color to color=(255, 255, 255) if working with grayscale images
            cv2.polylines(img=img_teeth_sides, pts=[tooth['right']['segments']], isClosed=False, color=(255, 0, 255), thickness=0, lineType=cv2.LINE_AA)  #note: set color to color=(255, 255, 255) if working with grayscale images

        ## Display Image
        #plt.figure(figsize=(20, 10))
        #image_rgb = cv2.cvtColor(img_teeth_sides,cv2.COLOR_BGR2RGB)
        #plt.imshow(image_rgb)
        ##plt.imshow(img_points, cmap='gray', vmin=0, vmax=255)
        #plt.show()
        #return image_rgb

    #image_rgb=showSideSegments(teeth)
    #cv2.imwrite('logs/3_1_teeth_sides.png', cv2.cvtColor(image_rgb,cv2.COLOR_RGB2BGR))
    
    #THE ONLY THING THAT DIFFERS FROM THE MAIN PROGRAM.
    #lengths=[]  <---  WE INITIALIZE THIS AT THE VERY BEGINNING SO TAHT WE ACCUMULATE THE RESULTS FOR ALL THE IMAGES
    SegmentPtA=RDP_contour[0][0]
    i=1
    while i<RDP_contour.shape[0]:
        SegmentPtB=RDP_contour[i][0]
        length=distance(SegmentPtA,SegmentPtB)
        lengths.append(length)
        SegmentPtA=SegmentPtB
        i=i+1
    ## Compute frequency and bins   ## WE DO THIS OUTSIDE OF THE MAIN LOOP!
    #frequency, bins = np.histogram(lengths, bins=20, range=[0, 100])
    ## Pretty Print
    #for b, f in zip(bins[1:], frequency):
    #    print(round(b, 1), ' '.join(np.repeat('*', f)))

    def erodeLeftSide(segments,threshold):
        SegmentPtA=segments[0][0]
        i=1
        foundLongSegment=False
        newSegments=[]
        while i<len(segments):
            SegmentPtB=segments[i][0]
            if foundLongSegment==False: #if we didn't yet found any long segment, check if the current one is long enought, otherwise this check is useless
                if distance(SegmentPtA,SegmentPtB)>threshold:
                    foundLongSegment = True
                    newSegments.append([SegmentPtA]) #as this is the fist segment being added, we also need to add point A. Point B will be added with the next if block
            if foundLongSegment==True: #if we already found a long segment keep adding segments (B point) till the end
                newSegments.append(segments[i])
            SegmentPtA=SegmentPtB
            i=i+1
        return np.array(newSegments)

    def erodeRightSide(segments,threshold):
        #do the same thig, but in reverse
        segments=np.flip(segments,0)
        segments=erodeLeftSide(segments,threshold)
        return np.flip(segments,0)

    #FILTER SEGMENTS IN THE DATA STRUCTURE
    #Note: technically for the following computations all we would really need are just the start point of the teeth on the 2 sides and the tip point (where segments change in direction)
    for tooth in teeth:
        tooth['left']['segments']=erodeLeftSide(tooth['left']['segments'],TEETH_SEGMENT_LEN_THRESH)
        tooth['right']['segments']=erodeRightSide(tooth['right']['segments'],TEETH_SEGMENT_LEN_THRESH)

    #REMOVE TEETH WITH NO SIDES LEFT
    #the previous filtering step might have removed all the segments of a side
    #(maybe one of the teeth side was almost out of frame, so the segment would be much shorter than normal)
    #so in that case remove the whole tooth (this is ok event in the case highlited above: too short side segments would produse imprecise results)
    newTeeth=[]
    for tooth in teeth:
        if len(tooth['left']['segments'])>=2 and len(tooth['right']['segments'])>=2: #segments have at leas 2 points
            newTeeth.append(tooth)
    teeth=newTeeth

    #VISUALIZE THE RESULT
    #image_rgb=showSideSegments(teeth)
    #cv2.imwrite('logs/4_1_filter_out_arcs.png', cv2.cvtColor(image_rgb,cv2.COLOR_RGB2BGR))
    
    #EXTRACT THE CONTOURS
    for tooth in teeth:
        tooth['left']['contour']=border_points_between(tooth['left']['segments'][0][0],tooth['left']['segments'][-1][0])
        tooth['right']['contour']=border_points_between(tooth['right']['segments'][0][0],tooth['right']['segments'][-1][0])
    
    #just some utility functions to calculate intersection between lines, angles between objects, etc...
    #line equation of the type y=(Δy/Δx)x+c=mx+c

    #@@@@@@@@@@@@@@@@@@@@@ LINES @@@@@@@@@@@@@@@@@@@@
    #calculate the function value at x given the line parameters slope (m) and intercept (c)
    def getY(m,c,x):
        return m*x+c

    #calculate the C constant giving a point of the line (x,y) and his slope (m)
    def getIntercept(x,y,m):
        return y-m*x

    #calculate the intersection point of two lines given theier slop (m1, m2) and intercept (c1, c2)
    def intersection_pt(m1, c1, m2, c2):
        x=-(c1-c2)/(m1-m2)
        y=m1*x+c1
        return [x,y]

    #calculate the intersection point of two lines defined by a collinear to the line (vx, vy) and a point of the line (x0,y0)
    def intersection_pt_cvlines(vx1,vy1,x01,y01,vx2,vy2,x02,y02):
        #lets handle some special cases first
        if vx1==0: #1st line slope would be infinite (Δx1=0): simply calculate y using 2nd line equation in the x on which lies the 1st (vertical) line
            x=x01
            y=getY(vy2/vx2,getIntercept(x02,y02,vy2/vx2),x)
            return [x,y]
        elif vx2==0: #2nd line slope would be infinite (Δx2=0): simply calculate y using 1st line equation in the x on which lies the 2nd (vertical) line
            x=x02
            y=getY(vy1/vx1,getIntercept(x01,y01,vy1/vx1),x)
            return [x,y]
        else:
            c1=getIntercept(x01,y01,vy1/vx1)
            m1=vy1/vx1
            c2=getIntercept(x02,y02,vy2/vx2)
            m2=vy2/vx2
            return intersection_pt(m1,c1,m2,c2)

    #intersection_pt_cvlines(0,0.5,2,2,0.5,1,-1,-1) #[2, 5]
    #intersection_pt_cvlines(2,1,1,1,1,2,3,0) #[4.33, 2.66]

    #@@@@@@@@@@@@@@@@@@@@ ANGLES @@@@@@@@@@@@@@@@@@@@
    def rad2deg(rad):
        return rad/math.pi*180
    def getSideAngle(vx,vy): #[0,+90] if it points down-right, ]0,-90] if down-left
        #if a vector points upward, invert a vector so that vercors that are pointing to I and II quadrants now points respectively to III and IIII ones (basically it roates them by 180° if needed)
        if vy>0:
            vx,vy=-vx,-vy
        if vx<0: #down-left
            return -(+90-rad2deg(math.atan(vy/vx))) # (atan>=0)
        else: #down-right
            return +90+rad2deg(math.atan(vy/vx)) #(atan<=0)

    #@@@@@@@@@@@@@@@@@@@@ PERPENDICULAR LINE given a line and a point @@@@@@@@@@@@@@@@@@@
    #this is useful to trace the segments approximating the sides of each tooth, specifically to make them of the right length
    #get the "m" and "c" values for a line passing through "(x,y)" and perpendicular to another line of slope "m"
    def perpendicularLine(x,y,m):
        newm=-1/m
        return [newm,y-newm*x]

    #given a line defined by "vx","vy","x","y" and a point defined by "j","k", get the point of the line with the minimum distance to point (j,k)
    def closestLinePt(vx,vy,x0,y0,j,k):
        #lets handle some special cases first
        if vx==0: #since the line is vertical take the the x coordinate of a point of the line ("x0"), and the y coordinate of the other point ("k")
            return [x0,k]
        elif vy==0: #since the horizontal take the the y coordinate of a point of the line ("y0"), and the x coordinate of the other point ("j")
            return [j,y0]
        else: #closest point is the intersection of the line and the perpendicular line passing through (j,k)
            m1=vy/vx
            c1=getIntercept(x0,y0,m1)
            m2,c2=perpendicularLine(j,k,m1)
            return intersection_pt(m1,c1,m2,c2)

    #closestLinePt(1,3,-1,1,1,2)  [-0.5, 2.5]
    #closestLinePt(1,0,-1,1,2,2)  [2, 1]
    #closestLinePt(0,1,-1,1,2,2)  [-1, 2]

    #@@@@@@@@@@@@@@@@@@ GENERAL @@@@@@@@@@@@@@@@@@
    def roundIntPt(pt):
        return [round(pt[0]),round(pt[1])]
    
    for tooth in teeth:
        tooth['left']['contour']=border_points_between(tooth['left']['segments'][0][0],tooth['left']['segments'][-1][0])
        tooth['right']['contour']=border_points_between(tooth['right']['segments'][0][0],tooth['right']['segments'][-1][0])

        tooth['left']['fitline_data']=cv2.fitLine(tooth['left']['contour'],cv2.DIST_L2,0,0.01,0.01)
        vx1,vy1,x01,y01=tuple(tooth['left']['fitline_data'].flatten())
        firstBorderPt=tooth['left']['contour'][0][0]
        firstLinePt=closestLinePt(vx1,vy1,x01,y01,firstBorderPt[0],firstBorderPt[1])
        tooth['left']['line_basepoint']=roundIntPt(firstLinePt)
        tooth['left']['angle']=getSideAngle(vx1,-vy1)

        tooth['right']['fitline_data']=cv2.fitLine(tooth['right']['contour'],cv2.DIST_L2,0,0.01,0.01)
        vx2,vy2,x02,y02=tuple(tooth['right']['fitline_data'].flatten())
        lastBorderPt=tooth['right']['contour'][-1][0]
        lastLinePt=closestLinePt(vx2,vy2,x02,y02,lastBorderPt[0],lastBorderPt[1])
        tooth['right']['line_basepoint']=roundIntPt(lastLinePt)
        tooth['right']['angle']=getSideAngle(vx2,-vy2)

        tooth['intersection']=intersection_pt_cvlines(vx1,vy1,x01,y01,vx2,vy2,x02,y02)
        tooth['angle']=abs(tooth['left']['angle']-tooth['right']['angle'])
    
    
    R=(37.0,42.5) #allowable_angle_range
    overlay_colors = {
      "red": (0, 0, 255),
      "green": (0, 255, 0)
    }

    #overlay on a new image the relevant data
    def overlay_teeth_data(img_to_overlay):
        for tooth in teeth:
            if tooth['angle']>=R[0] and tooth['angle']<=R[1]:
                color="green"
            else:
                color="red"
            color=overlay_colors[color]
            angleText=str(round(tooth['angle'],1))
            textPos=(round(tooth['intersection'][0]-10),round(tooth['intersection'][1]-14))
            img_to_overlay = cv2.putText(img=img_to_overlay, text=angleText, org=textPos, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=1, lineType=cv2.LINE_AA)
            cv2.line(img_to_overlay, tuple(tooth['left']['line_basepoint']), tuple(tooth['intersection']), color=color, thickness=1, lineType=cv2.LINE_AA)
            cv2.line(img_to_overlay, tuple(tooth['right']['line_basepoint']), tuple(tooth['intersection']), color=color, thickness=1, lineType=cv2.LINE_AA)
        return img_to_overlay

    img_overlayed = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img_overlayed=overlay_teeth_data(img_overlayed)

    ## Display Image
    #plt.figure(figsize=(20, 10))
    #image_rgb = cv2.cvtColor(img_overlayed,cv2.COLOR_BGR2RGB)
    #plt.imshow(image_rgb)
    ##plt.imshow(img_points, cmap='gray', vmin=0, vmax=255)
    #plt.show()

    cv2.imwrite('img/saved_results/final_result_{}.png'.format(imgNum), img_overlayed)
    
        
# Compute frequency and bins
frequency, bins = np.histogram(lengths, bins=40, range=[0, 100])
# Pretty Print
for b, f in zip(bins[1:], frequency):
    print(round(b, 1), ' '.join(np.repeat('*', f)))


Chosen treshold value:  123.0
Start point  (0, 509)  found, index  2060
Finish point  (1279, 444)  found, index  160
Chosen treshold value:  128.0
Start point  (0, 448)  found, index  1980
Finish point  (1279, 473)  found, index  51
Chosen treshold value:  125.0
Start point  (0, 377)  found, index  2002
Finish point  (1279, 391)  found, index  34
Chosen treshold value:  123.0
Start point  (0, 467)  found, index  2732
Finish point  (1279, 532)  found, index  900
Chosen treshold value:  123.0
Start point  (0, 543)  found, index  2339
Finish point  (1279, 472)  found, index  504
Chosen treshold value:  124.0
Start point  (0, 530)  found, index  2717
Finish point  (1279, 464)  found, index  872
Chosen treshold value:  124.0
Start point  (0, 487)  found, index  2002
Finish point  (1279, 428)  found, index  141
2.5 
5.0 * * *
7.5 * * * * * * * * * *
10.0 * * * * * * * * * * * * * * * * * *
12.5 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
15.0 * * * * *